In [ ]:
!pip3 install torch==1.13.0 torchvision torchaudio torchtext

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
import zipfile, os, urllib.request, glob, math, shutil
import torchvision
import torch.optim as optim
from torchvision import datasets, models, transforms
from sklearn.metrics import confusion_matrix, balanced_accuracy_score, precision_recall_fscore_support, roc_auc_score
import sklearn.metrics
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
import PIL
import PIL.Image

torch.manual_seed(0)

sys.path.append('drive/MyDrive')
import modelArchs

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

Mounted at /content/drive


In [ ]:
print(torch.__version__)
print(torchvision.__version__)

1.13.0+cu117
0.14.0+cu117


Χρησιμοποιώ όλα τα δεδομένα για το τελικό train.

In [ ]:
all_dataset = pd.read_csv('drive/MyDrive/app_metadata.csv')
all_dataset.head()

,patient_id,lesion_id,smoke,drink,background_father,background_mother,age,pesticide,gender,skin_cancer_history,...,diameter_2,diagnostic,itch,grew,hurt,changed,bleed,elevation,img_id,biopsed
0,PAT_1516,1765,NaN,NaN,NaN,NaN,8,NaN,NaN,NaN,...,NaN,NEV,False,False,False,False,False,False,PAT_1516_1765_530.png,False
1,PAT_46,881,False,False,POMERANIA,POMERANIA,55,False,FEMALE,True,...,5.0,BCC,True,True,False,True,True,True,PAT_46_881_939.png,True
2,PAT_1545,1867,NaN,NaN,NaN,NaN,77,NaN,NaN,NaN,...,NaN,ACK,True,False,False,False,False,False,PAT_1545_1867_547.png,False
3,PAT_1989,4061,NaN,NaN,NaN,NaN,75,NaN,NaN,NaN,...,NaN,ACK,True,False,False,False,False,False,PAT_1989_4061_934.png,False
4,PAT_684,1302,False,True,POMERANIA,POMERANIA,79,False,MALE,True,...,5.0,BCC,True,True,False,False,True,True,PAT_684_1302_588.png,True


In [ ]:
diagnostics = sorted(all_dataset.diagnostic.unique().tolist())

def create_label_num(x):
  return diagnostics.index(x)

all_dataset['diagnostic'] = all_dataset['diagnostic'].apply(create_label_num)
all_dataset.head()

,patient_id,lesion_id,smoke,drink,background_father,background_mother,age,pesticide,gender,skin_cancer_history,...,diameter_2,diagnostic,itch,grew,hurt,changed,bleed,elevation,img_id,biopsed
0,PAT_1516,1765,NaN,NaN,NaN,NaN,8,NaN,NaN,NaN,...,NaN,3,False,False,False,False,False,False,PAT_1516_1765_530.png,False
1,PAT_46,881,False,False,POMERANIA,POMERANIA,55,False,FEMALE,True,...,5.0,1,True,True,False,True,True,True,PAT_46_881_939.png,True
2,PAT_1545,1867,NaN,NaN,NaN,NaN,77,NaN,NaN,NaN,...,NaN,0,True,False,False,False,False,False,PAT_1545_1867_547.png,False
3,PAT_1989,4061,NaN,NaN,NaN,NaN,75,NaN,NaN,NaN,...,NaN,0,True,False,False,False,False,False,PAT_1989_4061_934.png,False
4,PAT_684,1302,False,True,POMERANIA,POMERANIA,79,False,MALE,True,...,5.0,1,True,True,False,False,True,True,PAT_684_1302_588.png,True


In [ ]:
all_dataset.diagnostic.value_counts()

1    845
0    730
3    244
5    235
4    192
2     52
Name: diagnostic, dtype: int64

Κατεβάζω τα δεδομένα.

In [ ]:
urllib.request.urlretrieve("https://prod-dcd-datasets-cache-zipfiles.s3.eu-west-1.amazonaws.com/zr7vgbcyr2-1.zip", "imgs.zip")

with zipfile.ZipFile('imgs.zip', 'r') as zip_ref:
  zip_ref.extractall('imgs_folder')

print("Done with unzipping...")

for i in ['imgs_folder/images/imgs_part_1.zip', 'imgs_folder/images/imgs_part_2.zip', 'imgs_folder/images/imgs_part_3.zip']:
  with zipfile.ZipFile(i, 'r') as zip_ref:
    zip_ref.extractall()

os.makedirs('images')

dest_folder = 'images'
for src_folder in ['imgs_part_1', 'imgs_part_2', 'imgs_part_3']:
  file_list = os.listdir(src_folder)
  print("Found %d files in folder: %s" %(len(file_list),src_folder))
  for filename in file_list:
    src_path = os.path.join(src_folder,filename)
    shutil.copy(src_path, dest_folder)

!ls images | wc -l

Done with unzipping...
Found 911 files in folder: imgs_part_1
Found 659 files in folder: imgs_part_2
Found 728 files in folder: imgs_part_3
2298


Αποθηκεύω κάθε εικόνα στον αντίστοιχο φάκελο.

In [ ]:
diagnostics_list = ['ACK', 'BCC', 'MEL', 'NEV', 'SCC', 'SEK']

def store_to_corresponding_folder(src_path_prefix,dest_path_prefix,current_dataset):

  #create folder for each dataset
  if os.path.exists(dest_path_prefix):
    print("Folder %s already exists..." %dest_path_prefix)
    return
  os.makedirs(dest_path_prefix)

  #create subfolder for each category...
  for i in diagnostics_list:
    folder_path = os.path.join(dest_path_prefix,i)
    os.makedirs(folder_path)

  #store each image to corresponding folder
  for index, row in current_dataset.iterrows():
    category_label = diagnostics_list[row['diagnostic']]
    img_name = row['img_id']

    src_path = os.path.join(src_path_prefix,img_name)
    dest_path = os.path.join(dest_path_prefix,category_label)
    shutil.copy(src_path,dest_path)

  #print number of elements in each folder
  print("For " + dest_path_prefix + ': ')
  for i in diagnostics_list:
    category_folder = os.path.join(dest_path_prefix,i)
    files_no = len(glob.glob(category_folder + '/*.png'))
    print(i + ': ' + str(files_no))

store_to_corresponding_folder('images','train_imgs', all_dataset)

For train_imgs: 
ACK: 730
BCC: 845
MEL: 52
NEV: 244
SCC: 192
SEK: 235


Ξεκινάω το train του μοντέλου σε όλα τα δεδομένα.

Αρχικά ορίζω το επιλεγμένο μοντέλο.

In [ ]:
model = modelArchs.get_MobileNetSa(6,True)
model.to(device)

Ορίζω διάφορα transformations για να κάνουμε augment το training set, ώστε να πετύχουμε καλύτερο generalization του μοντέλου.

In [ ]:
transforms_train = transforms.Compose([
    transforms.RandomResizedCrop(size = 224, scale = (0.4,1.0)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    transforms.ColorJitter(brightness = [0.7, 1.3], contrast = [0.7, 1.3], saturation = [0.7, 1.3], hue = [-0.1,0.1]),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

Ορισμός υπερ-παραμέτρων.

In [ ]:
epochs = 20
batch_len = 32

criterion = nn.CrossEntropyLoss()
learning_rate = 0.001
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size = 5, gamma=0.2)

Εδώ ορίζω τα dataset και dataloaders, χρησιμοποιώντας το weightRandomSampler για να δημιουργήσει balanced batches:

https://towardsdatascience.com/demystifying-pytorchs-weightedrandomsampler-by-example-a68aceccb452

In [ ]:
def get_weights_for_imgs(images, n_categories):

    n_images = len(images)

    #εδώ θα μπορούσα να χρησιμοποιήσω έτοιμους τους αριθμούς images per class από το train_dataset.category.value_counts() , ωστόσο στην συνέχεια όταν κάνω enumerate(images) στην
    #τελευταία for, αυτό μπορεί να δημιουργήσει πρόβλημα, καθώς δεν ξέρουμε το image_category σε ποιό label πραγματικά αντιστοιχεί...
    imgs_per_category = [0] * n_categories
    for _, image_category in images:
      imgs_per_category[image_category] += 1

    weight_per_category = [0] * n_categories # έχουμε 6 κατηγορίες
    for i in range(n_categories):
      weight_per_category[i] = 1 / float(imgs_per_category[i])#float(n_images) / float(imgs_per_category[i])

    weights = [0] * n_images
    for idx, (_, image_category) in enumerate(images):
        weights[idx] = weight_per_category[image_category]

    return weights

train_dataset = datasets.ImageFolder('train_imgs', transforms_train)

len_train_dataset = len(train_dataset)

print("Length of train_dataset: ", len_train_dataset)

weights = get_weights_for_imgs(train_dataset.imgs, len(train_dataset.classes))
weights = torch.DoubleTensor(weights)
train_sampler = torch.utils.data.sampler.WeightedRandomSampler(weights, len(weights))

train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size = batch_len, sampler = train_sampler, pin_memory=True)

Length of train_dataset:  2298


Το βασικό train loop.

In [ ]:
for epoch in range(1,epochs):

  model.train()
  train_running_loss = 0

  #Training phase
  print("Starting epoch: " , epoch)

  for i, (inputs, labels) in enumerate(train_dataloader,1):
      inputs = inputs.to(device)
      labels = labels.to(device)

      optimizer.zero_grad()
      outputs = model(inputs)
      loss = criterion(outputs, labels)
      loss.backward()
      optimizer.step()

      train_running_loss += loss.detach().item()

  print('Epoch: %d | Loss: %.4f ' %(epoch, train_running_loss / i))
  scheduler.step()

try:
  model.eval()
  checkpoint = {'model': model.state_dict()}
  checkpoint_fn = 'drive/MyDrive/final_app_model.pt'
  torch.save(checkpoint, checkpoint_fn)
except:
  print("Error in writing general checkpoint")